In [31]:
from multiprocessing import Pool
from joblib import Parallel, delayed
import multiprocessing
import pandas as pd
import re


data = pd.read_csv('Corea.csv', engine='python')

ADRframe=data[["#0Mid","14agree-disagree"]]

In [32]:
ADRframe.head(10)

,#0Mid,14agree-disagree
0,18006,NaN
1,18031,1.0
2,18086,1.0
3,18087,1.0
4,18106,1.0
5,18124,1.0
6,18145,NaN
7,18147,-1.0
8,18160,1.0
9,18179,-1.0


In [78]:
Lexframe = pd.read_csv('ItaloLexRate10.csv', engine='python')

In [79]:
Lexframe.head(10)

,cellaverage,mid
0,0.000000,18006
1,0.000000,18031
2,0.000000,18086
3,0.666667,18087
4,0.500000,18106
5,0.333333,18124
6,0.000000,18145
7,0.125000,18147
8,0.000000,18160
9,0.500000,18179


In [80]:
merged=pd.merge(Lexframe, ADRframe, left_on='mid', right_on='#0Mid')


In [81]:
merged.head(5)

,cellaverage,mid,#0Mid,14agree-disagree
0,0.000000,18006,18006,NaN
1,0.000000,18031,18031,1.0
2,0.000000,18086,18086,1.0
3,0.666667,18087,18087,1.0
4,0.500000,18106,18106,1.0


In [82]:
merged=merged.drop('mid', axis=1)

In [83]:
merged.head(5)

,cellaverage,#0Mid,14agree-disagree
0,0.000000,18006,NaN
1,0.000000,18031,1.0
2,0.000000,18086,1.0
3,0.666667,18087,1.0
4,0.500000,18106,1.0


In [84]:
mergedOrd=merged[["#0Mid","cellaverage","14agree-disagree"]]

In [85]:
mergedOrd.head()

,#0Mid,cellaverage,14agree-disagree
0,18006,0.000000,NaN
1,18031,0.000000,1.0
2,18086,0.000000,1.0
3,18087,0.666667,1.0
4,18106,0.500000,1.0


In [86]:
mergedOrd.to_csv("AWS10merg.csv")

In [87]:
mergedOrd=mergedOrd.sort_values(by='#0Mid')
mergedOrd=mergedOrd.dropna()
mergedOrd.head(10)

,#0Mid,cellaverage,14agree-disagree
1,18031,0.000000,1.0
2,18086,0.000000,1.0
3,18087,0.666667,1.0
4,18106,0.500000,1.0
5,18124,0.333333,1.0
7,18147,0.125000,-1.0
8,18160,0.000000,1.0
9,18179,0.500000,-1.0
10,18189,0.000000,1.0
11,18191,0.000000,-1.0


In [88]:
alist=[]
for index, cell in mergedOrd.iterrows():
    if (cell["cellaverage"])<=(-0.1):
        alist.append(-1)
    if (cell["cellaverage"])>=(0.1):
        alist.append(1)
    else:
        alist.append(0)



In [89]:
row1=pd.DataFrame()
row1["ADRpred"]=alist
output=pd.concat([mergedOrd, row1],axis=1)
output=output.dropna()
output.head(5)

,#0Mid,cellaverage,14agree-disagree,ADRpred
1,18031.0,0.000000,1.0,0.0
2,18086.0,0.000000,1.0,1.0
3,18087.0,0.666667,1.0,1.0
4,18106.0,0.500000,1.0,1.0
5,18124.0,0.333333,1.0,1.0


In [90]:
output=output.dropna()
output = output[output["14agree-disagree"] != 0]
output = output[output["cellaverage"] != 0]
output = output[output["ADRpred"] != 0]
output.head()

,#0Mid,cellaverage,14agree-disagree,ADRpred
3,18087.0,0.666667,1.0,1.0
4,18106.0,0.500000,1.0,1.0
5,18124.0,0.333333,1.0,1.0
7,18147.0,0.125000,-1.0,1.0
13,18273.0,0.500000,1.0,-1.0


In [91]:
from sklearn import metrics
metrics.mean_absolute_error(output["cellaverage"], output["14agree-disagree"])

0.95864099859353502

In [92]:
from sklearn.metrics import confusion_matrix

confusion_matrix(output["ADRpred"], output["14agree-disagree"])

array([[31, 27],
       [50, 50]])

In [93]:
metrics.mean_absolute_error(output["ADRpred"], output["14agree-disagree"])

0.97468354430379744

In [94]:
metrics.accuracy_score(output["14agree-disagree"],output["ADRpred"], )

0.51265822784810122

In [76]:
output.to_csv("preBinary.csv", index=False)

In [77]:
output2 = pd.read_csv('postBinary.csv', engine='python')


In [46]:
metrics.mean_absolute_error(output2["cellaverage"], output2["14agree-disagree"])

0.97585843873036082

In [47]:
metrics.accuracy_score(output2["14agree-disagree"],output2["ADRpred"], )

0.50743099787685775